In [1]:
import pandas as pd
import numpy as np
import plotly as plt

In [2]:
# pull shot data - note that this is from github - https://github.com/sealneaward/nba-movement-data

raw = pd.read_csv('shot_data.csv')
dropped = raw.drop(columns = ['EVENT_TYPE', 'GAME_EVENT_ID', 'GAME_DATE', 'GAME_ID'])

dropped.head(10)

,ACTION_TYPE,EVENTTIME,GRID_TYPE,HTM,LOC_X,LOC_Y,MINUTES_REMAINING,PERIOD,PLAYER_ID,PLAYER_NAME,...,SHOT_DISTANCE,SHOT_MADE_FLAG,SHOT_TIME,SHOT_TYPE,SHOT_ZONE_AREA,SHOT_ZONE_BASIC,SHOT_ZONE_RANGE,TEAM_ID,TEAM_NAME,VTM
0,Jump Shot,215,Shot Chart Detail,LAL,-23.9825,157.0968,3,1,101138,Brandon Bass,...,15,0,218.50,2PT Field Goal,Center(C),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,OKC
1,Dunk Shot,191,Shot Chart Detail,LAL,224.3221,24.9662,3,1,101138,Brandon Bass,...,0,1,204.83,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,OKC
2,Jump Shot,518,Shot Chart Detail,LAL,-5.4122,173.0601,8,2,101138,Brandon Bass,...,16,1,520.65,2PT Field Goal,Center(C),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,OKC
3,Jump Shot,120,Shot Chart Detail,LAL,-25.9351,158.7570,2,3,101138,Brandon Bass,...,15,0,123.26,2PT Field Goal,Center(C),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,OKC
4,Jump Shot,646,Shot Chart Detail,LAL,178.3350,20.1712,10,1,201579,Roy Hibbert,...,15,1,650.49,2PT Field Goal,Right Side(R),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,OKC
5,Hook Shot,552,Shot Chart Detail,LAL,100.3073,98.9265,9,1,201579,Roy Hibbert,...,5,0,546.23,2PT Field Goal,Center(C),In The Paint (Non-RA),Less Than 8 ft.,1610612747,Los Angeles Lakers,OKC
6,Hook Shot,430,Shot Chart Detail,LAL,111.9160,222.6307,7,1,201579,Roy Hibbert,...,4,0,449.90,2PT Field Goal,Center(C),In The Paint (Non-RA),Less Than 8 ft.,1610612747,Los Angeles Lakers,OKC
7,Hook Shot,590,Shot Chart Detail,LAL,-92.6563,65.3523,9,3,201579,Roy Hibbert,...,5,0,593.22,2PT Field Goal,Center(C),In The Paint (Non-RA),Less Than 8 ft.,1610612747,Los Angeles Lakers,OKC
8,Jump Shot,401,Shot Chart Detail,LAL,-216.1135,-10.5376,6,1,1626204,Larry Nance Jr.,...,13,1,408.84,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,OKC
9,Layup Shot,273,Shot Chart Detail,LAL,-219.9258,16.3056,4,2,1626204,Larry Nance Jr.,...,1,0,297.98,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,OKC


Get the shots grouped by player in a dictionary

In [ ]:

playerDict = {} # first is player name, second is their shots

# get list of ids
playerIds = dropped.PLAYER_ID.unique()

# make dictionary with empty arrays for each player's shots
for pid in playerIds:
    playerDict[pid] = []

# for each shot, add it to the player ID's array
for index, row in dropped.iterrows():
    innerPid = row['PLAYER_ID']
    shotMade = row['SHOT_MADE_FLAG']
    
    playerDict[innerPid].append(shotMade)

In [4]:
# try 2
playerAvgGrouped = dropped.groupby('PLAYER_ID', as_index=False)['SHOT_MADE_FLAG'].mean()
hm = playerAvgGrouped.set_index(playerAvgGrouped['PLAYER_ID'])
print('hello')

hello


In [ ]:
# copy
full = dropped.sample(frac = 1)
full['PLAYER_AVG'] = ''

# add average
for index, row in full.iterrows():
    row['PLAYER_AVG'] = hm.loc[row['PLAYER_ID']]['SHOT_MADE_FLAG']
    
full

In [ ]:
shotAvgDict = {}

# get average of python list
def avg(l): 
    return sum(l) / len(l) 

for p in playerDict.keys():
    shotAvgDict[p] = avg(playerDict[p])

In [7]:
# copy
full = dropped.sample(frac = 1)
full['PLAYER_AVG'] = ''

# add average
for index, row in full.iterrows():
    row['PLAYER_AVG'] = shotAvgDict[row['PLAYER_ID']]

ACTION_TYPE                        Jump Shot
EVENTTIME                                156
GRID_TYPE                  Shot Chart Detail
HTM                                      IND
LOC_X                                -114.33
LOC_Y                                223.428
MINUTES_REMAINING                          2
PERIOD                                     1
PLAYER_ID                             203110
PLAYER_NAME                   Draymond Green
QUARTER                                    1
SECONDS_REMAINING                         36
SHOT_ATTEMPTED_FLAG                        1
SHOT_DISTANCE                             26
SHOT_MADE_FLAG                             0
SHOT_TIME                             158.39
SHOT_TYPE                     3PT Field Goal
SHOT_ZONE_AREA          Left Side Center(LC)
SHOT_ZONE_BASIC            Above the Break 3
SHOT_ZONE_RANGE                      24+ ft.
TEAM_ID                           1610612744
TEAM_NAME              Golden State Warriors
VTM       